# 准备方法

In [7]:
import jieba
from collections import Counter
from pathlib import Path
from tqdm import tqdm
import os
import sys

sys.path.append('../')

from examination_paper_classifier import extract_text
from examination_paper_classifier import is_all_chinese_or_english
from examination_paper_category_classifier import classify_paper_type
from examination_paper_category_classifier import CLASSIFY_KEYWORDS


def most_common_words(text, n=40):
    words = jieba.cut(text)
    words = filter(lambda word: len(word) > 1 and is_all_chinese_or_english(word), words)
    word_count = Counter(words)
    most_common = word_count.most_common(n)
    most_common_dict = dict(most_common)
    return most_common_dict


def add_dicts(dicts_list):
    result_dict = {}
    for d in dicts_list:
        for key, value in d.items():
            if key in result_dict:
                result_dict[key] += value
            else:
                result_dict[key] = value
            
    return sorted(result_dict.items(), key=lambda x: x[1], reverse=True)


def search_most_keywords_by_file(file_path):
    file_path = Path(file_path)
    try:
        text = extract_text(str(file_path), file_path.suffix)
        return most_common_words(text)
    except:
        return {}
    
    
def search_most_keywords_by_dir(dir_path, file_name_keyword=[]):
    dicts_list = []
    for file_path in tqdm(list(Path(dir_path).rglob('*.doc*'))):
        file_name = file_path.name
        if not file_name_keyword:
            dicts_list.append(search_most_keywords_by_file(file_path))
        elif any([word in file_name for word in file_name_keyword]):
            dicts_list.append(search_most_keywords_by_file(file_path))
            
    return add_dicts(dicts_list)

# 首先查看各种类型试卷所需要的关键词

In [32]:
all_categorys_total = {"化学":[], "历史":[], "地理":[], "政治":[], "数学":[], "物理":[], "生物":[], "语文":[] }


for file_path in tqdm(list(Path("../data/docx/题库").rglob('*.doc*'))):
    file_name = file_path.name
    
    for category in all_categorys_total:
        if category in file_name:
            all_categorys_total[category].append(search_most_keywords_by_file(file_path))

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8298/8298 [02:52<00:00, 48.04it/s]


In [3]:
for category in all_categorys_total:
    keywords_total = add_dicts(all_categorys_total[category])[:20]
    print(category)
    print(keywords_total)
    print()

化学
[('反应', 12393), ('pic', 11694), ('溶液', 8524), ('生成', 4338), ('正确', 3100), ('下列', 3069), ('mol', 3069), ('元素', 2812), ('物质', 2766), ('发生', 2744), ('离子', 2621), ('答案', 2377), ('方程式', 2218), ('OH', 1959), ('分析', 1857), ('氧化', 1826), ('结构', 1776), ('实验', 1768), ('加入', 1616), ('化学', 1612)]

历史
[('材料', 2324), ('中国', 1491), ('答案', 968), ('历史', 893), ('发展', 887), ('根据', 810), ('所学', 802), ('结合', 799), ('经济', 735), ('知识', 718), ('国家', 674), ('社会', 493), ('世纪', 486), ('世界', 466), ('美国', 464), ('政治', 440), ('英国', 331), ('主要', 325), ('要求', 322), ('问题', 316)]

地理
[('pic', 977), ('主要', 948), ('地区', 826), ('发展', 721), ('我国', 657), ('城市', 598), ('影响', 583), ('回答', 480), ('原因', 425), ('形成', 405), ('分布', 389), ('答案', 370), ('完成', 358), ('问题', 348), ('变化', 343), ('经济', 325), ('人口', 305), ('材料', 283), ('分析', 272), ('错误', 251)]

政治
[('发展', 2362), ('文化', 1683), ('材料', 1354), ('中国', 1339), ('经济', 1314), ('社会', 1094), ('我国', 1090), ('国家', 994), ('企业', 761), ('问题', 659), ('人民', 658), ('政府', 626), ('答案', 600

# 读取测试数据看是否存在偏差(运行完分类脚本后)

In [31]:
import pandas as pd

all_file_df = pd.read_csv("../category_classifier.csv")

all_file_df

,file_path,target_path,probability,categorie
0,./data/docx/题库/生物/高中/数学(2)/高中/高考真题【391.82MB】/2...,./data/docx/题库/生物/高中/数学(2)/高中/高考真题【391.82MB】/2...,1.000000,生物
1,./data/docx/题库/生物/高中/jiaocai/教材/学校教育（电子教材）/高中学...,./data/docx/题库/生物/高中/jiaocai/教材/学校教育（电子教材）/高中学...,1.000000,生物
2,./data/docx/题库/生物/初中/小品/按合集分/【科教类】/【初中思维导图】_9科...,./data/docx/题库/生物/初中/小品/按合集分/【科教类】/【初中思维导图】_9科...,0.880647,生物
3,./data/docx/题库/数学/小学/其他/小学-1-6年级试卷合集--25.3 GB/...,./data/docx/题库/数学/小学/其他/小学-1-6年级试卷合集--25.3 GB/...,0.880647,语文
4,./data/docx/题库/数学/小学/其他/小学-1-6年级试卷合集--25.3 GB/...,./data/docx/题库/数学/小学/其他/小学-1-6年级试卷合集--25.3 GB/...,1.000000,数学
...,...,...,...,...
10141,./data/docx/202305/全网首发！2022最新考研资料合集！【1.8TB】/历...,./data/docx/202305/全网首发！2022最新考研资料合集！【1.8TB】/历...,0.789423,政治
10142,./data/docx/202305/全网首发！2022最新考研资料合集！【1.8TB】/保...,./data/docx/202305/全网首发！2022最新考研资料合集！【1.8TB】/保...,0.799960,生物
10143,./data/docx/202305/电子书大合集/各种PDF格式书籍/建筑类书籍/建筑类书...,./data/docx/202305/电子书大合集/各种PDF格式书籍/建筑类书籍/建筑类书...,0.968779,indefinable
10144,./data/docx/202305/电子书大合集/各种PDF格式书籍/建筑类书籍/建筑类书...,./data/docx/202305/电子书大合集/各种PDF格式书籍/建筑类书籍/建筑类书...,0.999801,indefinable


In [32]:
len(all_file_df)

10146

In [33]:
all_file_df['categorie'].value_counts()

数学             3694
公务员            1375
医学             1206
化学              600
地理              576
语文              557
物理              488
政治              421
历史              248
生物              235
机械自动化           221
indefinable     162
other           138
理综               66
文综               63
面试               55
哲学               29
自选                8
心理                4
Name: categorie, dtype: int64

# 通过文件路径以及文件名验证数据准确性

In [36]:
class_name = "indefinable"

print(len(all_file_df[(all_file_df["categorie"] == class_name) & ~all_file_df["file_path"].str.contains(class_name)]))

for _, row in all_file_df.iterrows():
    if row['categorie'] == class_name and class_name not in row["file_path"]:
        print(f"{row['categorie']} {row['file_path']}")
        print()

162
indefinable ./data/docx/202307/【09】🏛️文化厅｜知识的海洋🌊(2)/🗿社科人文/【文化艺术】/乐理/乐理试题.doc

indefinable ./data/docx/202307/【09】🏛️文化厅｜知识的海洋🌊(2)/🗿社科人文/【文化艺术】/乐理/简谱乐理知识.doc

indefinable ./data/docx/202307/2022(1).最新整理超精品考证+资料大合集！（7TB+）/注册会计师 考试资料/2021年中级社会工作资格考试辅导资料/社会工作综合能力真题解析/2018.doc

indefinable ./data/docx/202307/2022(1).最新整理超精品考证+资料大合集！（7TB+）/注册会计师 考试资料/2021年中级社会工作资格考试辅导资料/社会工作综合能力真题解析/2019.doc

indefinable ./data/docx/202307/10TB精整资料合集【公众号：锦技社】/｜ 精选电子书籍 ｜ @【公众号：锦技社】/_ 精选电子书籍合集 _/掌盟☯️研报中心重磅9月29日更新【内部流出】/01掌盟☯️商业计划书与可行性报告大全/15、物流交通商业计划书/【公益知识库zscc.club】XX物流公司商业计划书.doc

indefinable ./data/docx/202307/10TB精整资料合集【公众号：锦技社】/｜ 精选电子书籍 ｜ @【公众号：锦技社】/_ 精选电子书籍合集 _/掌盟☯️研报中心重磅9月29日更新【内部流出】/01掌盟☯️商业计划书与可行性报告大全/《商业计划》社群会员专属资料请勿外传（原盘）/14、通讯、广告IT可行性报告/XX网成立的商业计划书.doc

indefinable ./data/docx/202307/10TB精整资料合集【公众号：锦技社】/｜ 精选电子书籍 ｜ @【公众号：锦技社】/_ 精选电子书籍合集 _/掌盟☯️研报中心重磅9月29日更新【内部流出】/01掌盟☯️商业计划书与可行性报告大全/32、通讯、广告IT可行性报告/【公益知识库zscc.club】XX网成立的商业计划书.doc

indefinable ./data/docx/202307/掌盟☯️书库大全（请下载后阅读不要在线浏览）(2)/23TB电子书合集/

# 在本地文件夹内搜索关键词的数量

In [79]:
search_most_keywords_by_dir("../data/docx/202307/公考状元笔记/【笔航公考】180元资料大礼包/2018-2019年公务员考试真题答案解析/", {})

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 33/33 [00:06<00:00,  5.16it/s]


[('答案', 8012),
 ('正确', 6144),
 ('排除', 5561),
 ('解析', 3965),
 ('符合', 3020),
 ('根据', 2188),
 ('选项', 2156),
 ('关系', 1787),
 ('可以', 1695),
 ('逻辑关系', 1682),
 ('错误', 1667),
 ('定义', 1658),
 ('没有', 1604),
 ('发展', 1563),
 ('本题', 1556),
 ('一致', 1496),
 ('可知', 1387),
 ('一个', 1384),
 ('当选', 1303),
 ('对应', 1295),
 ('通过', 1255),
 ('横线', 1149),
 ('我国', 1107),
 ('问题', 947),
 ('中国', 929),
 ('进行', 891),
 ('与题', 762),
 ('一种', 732),
 ('文化', 731),
 ('强调', 699),
 ('出处', 608),
 ('下列', 595),
 ('文意', 573),
 ('全国', 572),
 ('材料', 565),
 ('搭配', 555),
 ('自己', 551),
 ('亿元', 547),
 ('增长', 537),
 ('无法', 524),
 ('主要', 506),
 ('属于', 489),
 ('城市', 488),
 ('不是', 435),
 ('文字', 391),
 ('技术', 386),
 ('pic', 372),
 ('产生', 371),
 ('产品', 370),
 ('体现', 337),
 ('地区', 332),
 ('收入', 301),
 ('说明', 287),
 ('二者', 280),
 ('世界', 278),
 ('企业', 269),
 ('社会', 258),
 ('经济', 243),
 ('行为', 225),
 ('图形', 219),
 ('定位', 219),
 ('研究', 210),
 ('需要', 208),
 ('题干', 203),
 ('完成', 202),
 ('论点', 200),
 ('就诊', 200),
 ('以上', 183),
 ('方式', 179),
 ('所以', 

In [117]:
file_path = "../data/docx/202307/公考状元笔记/【笔航公考】180元资料大礼包/2018-2019年公务员考试真题试卷/福建卷.docx"
search_most_keywords_by_file(file_path)

{'下列': 37,
 '发展': 32,
 '一个': 27,
 '填入': 26,
 '我们': 25,
 '全国': 24,
 '部分': 24,
 '一项': 23,
 '横线': 22,
 '恰当': 22,
 '其中': 21,
 '影响': 21,
 '我国': 20,
 '研究': 20,
 '汽车': 20,
 '文字': 19,
 '依次': 19,
 '演出': 19,
 '如果': 17,
 '一种': 16,
 '没有': 16,
 '发现': 16,
 '微生物': 16,
 '世界': 15,
 '选项': 15,
 '人们': 15,
 '食品': 15,
 '主要': 15,
 '可以': 15,
 '艺术表演': 15,
 '正确': 14,
 '可能': 14,
 '医生': 14,
 '一定': 14,
 '电影': 14,
 '参加': 14,
 '增长': 14,
 '国家': 13,
 '自己': 13,
 '使用': 13}

In [118]:
# file_path = "../data/docx/202307/机械、自动化和电类专业课/机械、自动化和电类专业课/微型计算机原理/╬ó╨═╝╞╦π╗·╘¡└φ ╩╘╛φ/微型计算机原理 试卷/微型计算机原理考卷B答案.doc"
classify_paper_type(extract_text(file_path, "."+file_path.split(".")[-1]), file_name=file_path.split("/")[-1], debug=True)

[('生物', 100), ('政治', 70), ('历史', 67), ('医学', 38), ('化学', 35), ('公务员', 34), ('地理', 32), ('数学', 21), ('物理', 21), ('机械自动化', 16), ('语文', 8), ('心理', 5), ('理综', 0), ('文综', 0), ('自选', 0), ('other', 0)]


'生物'

In [12]:
def simple_judgment_paper_category_by_path(file_path):
    paper_types = [['公务员', '公考', '行测', '考公'], ['医学'], ['化学'], ['历史'], ['地理'], ['政治'], ['数学', '奥数'], ['物理'], ['生物'], ['语文'], ['心理'], ['机械自动化', '机械'], ['理综', '理科综合'], ['文综', '文科综合'], ['自选'], ['other'], ['面试'], ['哲学', '马哲', '马原']]

    last_occurrence = {paper_type[0]: -1 for paper_type in paper_types}  # Initialize last occurrence indices

    # Iterate over each paper type and its keywords
    for paper_type in paper_types:
        for keyword in paper_type:
            index = file_path.rfind(keyword)  # Find the last occurrence of this keyword
            if index > last_occurrence[paper_type[0]]:  # If this occurrence is later than the current one
                last_occurrence[paper_type[0]] = index  # Update the last occurrence index

    # Find the paper type with the maximum last occurrence index
    max_index_paper_type = max(last_occurrence, key=last_occurrence.get)

    # If the maximum index is -1, it means no paper type was found
    if last_occurrence[max_index_paper_type] == -1:
        return None
    else:
        return max_index_paper_type


In [13]:
simple_judgment_paper_category_by_path("./data/docx/docx/20230608002026_mXLQ2nPH8g-jnz9q/00：【2023年持续更新中】各类热点专题活动汇总/【2023年5月份更新】各类汇编/2023年5月份/5月每日更新/5月27日已发打包/公考遴选每日考题10道（2023年5月23日）【官方抖音号：资深秘书】.docx")

'公务员'